In [1]:

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:

df2017 = pd.read_excel("DatosMunicipalesGeneral_2017.xlsx")
df2018= pd.read_excel("DatosMunicipalesGeneral_2018.xlsx")
df2019= pd.read_excel("DatosMunicipalesGeneral_2019.xlsx")
df2020= pd.read_excel("DatosMunicipalesGeneral_2020.xlsx")
df2021= pd.read_excel("DatosMunicipalesGeneral_2021.xlsx")
df2022= pd.read_excel("DatosMunicipalesGeneral_2022.xlsx")

In [3]:
import pandas as pd

def procesar_datos_geograficos(df, año):
    # Añadir la columna de año
    df["Año"] = año


    # Asegurar que Latitud y Longitud sean cadenas para poder reemplazar comas por puntos
    df["Latitud"] = df["Latitud"].astype(str).str.replace(",", ".")
    df["Longitud"] = df["Longitud"].astype(str).str.replace(",", ".")

    # Convertir las cadenas modificadas a valores flotantes
    df["Latitud"] = df["Latitud"].astype(float)
    df["Longitud"] = df["Longitud"].astype(float)

    # Eliminar filas donde la latitud o la longitud sean NaN
    df.dropna(subset=["Latitud", "Longitud"], inplace=True)
    if 'N Accidentes con Victimas' in df.columns:
        df.dropna(subset=["N Accidentes con Victimas"], inplace=True)
    
    # Definir los límites de latitud y longitud para incluir toda España
    lat_min, lat_max = 27.0, 44.0  # Extender el máximo hacia el norte para incluir todo
    lon_min, lon_max = -18.0, 5.0  # Ajustar si es necesario para capturar todo el este

    # Filtrar el DataFrame por latitud y longitud dentro de estos nuevos límites
    df_filtrado = df[(df["Latitud"] >= lat_min) & (df["Latitud"] <= lat_max) &
                    (df["Longitud"] >= lon_min) & (df["Longitud"] <= lon_max)]

    

    return df_filtrado

In [4]:
df2017=procesar_datos_geograficos(df2017, 2017)
df2018=procesar_datos_geograficos(df2018, 2018)
df2019=procesar_datos_geograficos(df2019, 2019)
df2020=procesar_datos_geograficos(df2020, 2020)
df2021=procesar_datos_geograficos(df2021, 2021)
df2022=procesar_datos_geograficos(df2022, 2022)



In [5]:
df2022.columns

Index(['Codigo INE', 'Municipio', 'Provincia', 'Comunidad Autonoma',
       'Poblacion Total', 'Poblacion Hombres', 'Poblacion Mujeres',
       'Conductores Hombres', 'Conductoras Mujeres', 'Censo Conductores',
       'Parque Ciclomotores', 'Parque Motocicletas', 'Parque Turismos',
       'Parque Furgonetas', 'Parque Camiones', 'Parque Total',
       'Motocicletas sin ITV (<25 años)', 'Turismos sin ITV (<25 años)',
       'Resto de Vehiculos sin ITV (<25 años)',
       'Parque Ciclomotores (<25 años)', 'Parque Ciclomotores (<15 años)',
       'Parque Ciclomotores (<8 años)', 'Parque Ciclomotores (<4 años)',
       'Parque Motocicletas (<25 años)', 'Parque Motocicletas (<15 años)',
       'Parque Motocicletas (<8 años)', 'Parque Motocicletas (<4 años)',
       'Parque Turismos (<25 años)', 'Parque Turismos (<15 años)',
       'Parque Turismos (<8 años)', 'Parque Turismos (<4 años)',
       'Parque Furgonetas (<25 años)', 'Parque Furgonetas (<15 años)',
       'Parque Furgonetas (<8 años

In [6]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Suponiendo que tienes los dataframes ya cargados y listos para ser usados
# df2017, df2018, df2019, df2020, df2021, df2022

# Crear la aplicación Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Distribución de Conductores por Género en España", style={'text-align': 'center', 'color': 'white'}),
    dcc.Dropdown(
        id='year-selector',
        options=[
            {'label': '2017', 'value': '2017'},
            {'label': '2018', 'value': '2018'},
            {'label': '2019', 'value': '2019'},
            {'label': '2020', 'value': '2020'},
            {'label': '2021', 'value': '2021'},
            {'label': '2022', 'value': '2022'}
        ],
        value='2017'  # Valor por defecto
    ),
    html.Div(id='map-container')
])

@app.callback(
    Output('map-container', 'children'),
    [Input('year-selector', 'value')]
)
def update_map(selected_year):
    df = globals()[f'df{selected_year}']  # Acceder al dataframe basado en el año seleccionado
    df['Diferencia'] = df['Conductoras Mujeres'] - df['Conductores Hombres']
    df['Leyenda'] = df.apply(lambda row: 'Más mujeres' if row['Diferencia'] > 0 else 'Más hombres', axis=1)

    # Crear figura con Plotly Express
    fig = px.scatter_mapbox(df, lat="Latitud", lon="Longitud", color="Leyenda",
                            color_discrete_map={"Más mujeres": "blue", "Más hombres": "red"},
                            hover_name="Leyenda",
                            hover_data={"Conductoras Mujeres": True, "Conductores Hombres": True, "Latitud": False, "Longitud": False, "Leyenda": False},
                            zoom=5)

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    return dcc.Graph(figure=fig)

# Iniciar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


In [7]:
import pandas as pd

def agruparPorComunidad(df):
    
    df_agrupado = df.groupby('Comunidad Autonoma').agg({
        'Parque Ciclomotores': 'sum',
        'Parque Motocicletas': 'sum',
        'Parque Turismos': 'sum',
        'Parque Furgonetas': 'sum',
        'Parque Camiones': 'sum'
    }).reset_index()
    
    return df_agrupado

df2017_agrupado = agruparPorComunidad(df2017)
df2018_agrupado = agruparPorComunidad(df2018)
df2019_agrupado = agruparPorComunidad(df2019)
df2020_agrupado = agruparPorComunidad(df2020)
df2021_agrupado = agruparPorComunidad(df2021)
df2022_agrupado = agruparPorComunidad(df2022)


In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objects as go

dataframes = [df2017_agrupado, df2018_agrupado, df2019_agrupado, df2020_agrupado, df2021_agrupado, df2022_agrupado]
for i, df in enumerate(dataframes):
    df['Año'] = 2017 + i

df_consolidado = pd.concat(dataframes)

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='year-selector',
        options=[{'label': year, 'value': year} for year in range(2017, 2023)],
        value=2017
    ),
    dcc.Graph(id='vehicles-distribution-graph')
])

@app.callback(
    Output('vehicles-distribution-graph', 'figure'),
    [Input('year-selector', 'value')]
)
def update_graph(selected_year):
    filtered_df = df_consolidado[df_consolidado['Año'] == selected_year]

    # Preparar datos para el gráfico
    traces = []
    for vehiculo in ['Parque Ciclomotores', 'Parque Motocicletas', 'Parque Turismos', 'Parque Furgonetas', 'Parque Camiones']:  # Asegúrate de que estos sean los nombres de tus columnas
        traces.append(go.Bar(
            x=filtered_df['Comunidad Autonoma'],
            y=filtered_df[vehiculo],
            name=vehiculo
        ))

    figure = {
        'data': traces,
        'layout': go.Layout(
            title='Distribución de Vehículos por Comunidad Autónoma',
            xaxis={'title': 'Comunidad Autónoma'},
            yaxis={'title': 'Cantidad de Vehículos'},
            barmode='stack'
        )
    }

    return figure

if __name__ == '__main__':
    app.run_server(debug=True)


In [10]:
import dash
from dash import html, dcc, Input, Output
import pandas as pd
import plotly.express as px

df_total = pd.concat([df2017, df2018, df2019, df2020, df2021, df2022])

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el layout de la aplicación
app.layout = html.Div([
    html.H1("Proporción de Vehículos por Rango de Antigüedad", style={'text-align': 'center', 'color': 'white'}),
    dcc.Dropdown(
        id='selector-año',
        options=[{'label': año, 'value': año} for año in range(2017, 2023)],
        value=2017, # Valor por defecto
        placeholder="Selecciona un año"
    ),
    dcc.Dropdown(
        id='selector-area',
        options=[
            {'label': i, 'value': i} for i in df_total['Comunidad Autonoma'].unique()
        ],
        value=df_total['Comunidad Autonoma'].unique()[0], # Valor por defecto
        placeholder="Selecciona una área"
    ),
    dcc.Dropdown(
        id='selector-tipo-vehiculo',
        options=[
            {'label': 'Ciclomotores', 'value': 'Ciclomotores'},
            {'label': 'Motocicletas', 'value': 'Motocicletas'},
            {'label': 'Turismos', 'value': 'Turismos'},
            {'label': 'Furgonetas', 'value': 'Furgonetas'},
            {'label': 'Camiones', 'value': 'Camiones'}
        ],
        value='Ciclomotores', # Valor por defecto
        placeholder="Selecciona un tipo de vehículo"
    ),
    dcc.Graph(id='grafico-torta-antiguedad')
])

# Callback para actualizar el gráfico de torta
@app.callback(
    Output('grafico-torta-antiguedad', 'figure'),
    [Input('selector-año', 'value'),
     Input('selector-area', 'value'),
     Input('selector-tipo-vehiculo', 'value')]
)
def actualizar_grafico(año_seleccionado, area_seleccionada, tipo_vehiculo_seleccionado):
    # Filtrar el DataFrame por año y área seleccionada
    df_filtrado = df_total[(df_total['Año'] == año_seleccionado) & (df_total['Comunidad Autonoma'] == area_seleccionada)]
    
    # Preparar los datos para el gráfico de torta
    columnas_antiguedad = [f'Parque {tipo_vehiculo_seleccionado} (<25 años)',
                           f'Parque {tipo_vehiculo_seleccionado} (<15 años)',
                           f'Parque {tipo_vehiculo_seleccionado} (<8 años)',
                           f'Parque {tipo_vehiculo_seleccionado} (<4 años)']
    valores = [df_filtrado[col].sum() for col in columnas_antiguedad]
    rangos = ['<25 años', '<15 años', '<8 años', '<4 años']
    
    # Crear el gráfico de torta
    fig = px.pie(names=rangos, values=valores, title=f'Proporción de {tipo_vehiculo_seleccionado} por Rango de Antigüedad en {area_seleccionada}, Año {año_seleccionado}')
    
    return fig

# Iniciar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


In [5]:
df_total = pd.concat([df2017, df2018, df2019, df2020, df2021, df2022])

In [11]:
import dash
from dash import html, dcc, Input, Output
import pandas as pd
import plotly.express as px

# Asumiendo que df_total ya contiene todos los datos y tiene las columnas 'Latitud' y 'Longitud'

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el layout de la aplicación
app.layout = html.Div([
    html.H1("Distribución de Vehículos por Ubicación", style={'text-align': 'center','color': 'white'}),
    dcc.Dropdown(
        id='selector-año',
        options=[{'label': año, 'value': año} for año in range(2017, 2023)],
        value=2017, # Valor por defecto
        placeholder="Selecciona un año"
    ),
    dcc.Dropdown(
        id='selector-area',
        options=[
            {'label': i, 'value': i} for i in df_total['Comunidad Autonoma'].unique()
        ],
        value=df_total['Comunidad Autonoma'].unique()[0], # Valor por defecto
        placeholder="Selecciona una área"
    ),
    dcc.Dropdown(
        id='selector-tipo-vehiculo',
        options=[
            {'label': 'Ciclomotores', 'value': 'Ciclomotores'},
            {'label': 'Motocicletas', 'value': 'Motocicletas'},
            {'label': 'Turismos', 'value': 'Turismos'},
            {'label': 'Furgonetas', 'value': 'Furgonetas'},
            {'label': 'Camiones', 'value': 'Camiones'}
        ],
        value='Ciclomotores', # Valor por defecto
        placeholder="Selecciona un tipo de vehículo"
    ),
    dcc.Graph(id='mapa-densidad')
])

# Callback para actualizar el mapa
@app.callback(
    Output('mapa-densidad', 'figure'),
    [Input('selector-año', 'value'),
     Input('selector-area', 'value'),
     Input('selector-tipo-vehiculo', 'value')]
)
def actualizar_mapa(año_seleccionado, area_seleccionada, tipo_vehiculo_seleccionado):
    # Filtrar el DataFrame por año y área seleccionada
    df_filtrado = df_total[(df_total['Año'] == año_seleccionado) & (df_total['Comunidad Autonoma'] == area_seleccionada)]
    
    # Preparar los datos para el mapa
    # Asumiendo que cada registro en df_filtrado es un punto a visualizar y queremos sumar todos los vehículos de ese tipo para el tamaño del círculo
    df_filtrado['Cantidad'] = df_filtrado[f'Parque {tipo_vehiculo_seleccionado}'].fillna(0)
    
    # Crear el mapa con círculos que representan la cantidad de vehículos
    fig = px.scatter_mapbox(df_filtrado, lat='Latitud', lon='Longitud', size='Cantidad',
                            color_discrete_sequence=["blue"], size_max=30, zoom=5,
                            mapbox_style="open-street-map",
                            title=f'Distribución de {tipo_vehiculo_seleccionado} en {area_seleccionada}, Año {año_seleccionado}')
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

# Iniciar el servidor
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\lolar\AppData\Local\Temp\ipykernel_7768\182800576.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lolar\AppData\Local\Temp\ipykernel_7768\182800576.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

